Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Setup AML Workspace
---

This notebook walks you through all the necessary steps to configure your environment for this solution accelerator including:

1. Connecting to your workspace 
2. Create a config.json (this can be skipped if running on a Notebook VM)


## 1.0 Connect to workspace

Connect this solution accelerator to your AML workspace. This step isn't necessary if you're using a Notebook VM.

The following cell allows you to specify your workspace parameters. This cell uses the python method os.getenv to read values from environment variables which is useful for automation. If no environment variable exists, the parameters will be set to the specified default values.

In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="westus2")

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id=subscription_id, 
                   resource_group=resource_group, 
                   workspace_name=workspace_name)
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except Exception as e:
    print(e)
    print("Workspace not accessible. Change your parameters or create a new workspace below")

In [ ]:
ws.get_details()

### 1.1 Create workspace if needed
If you don't have a workspace already, uncomment the lines below to create one:

In [ ]:
# Create the workspace using the specified parameters
# ws = Workspace.create(
#    name=workspace_name,
#    subscription_id=subscription_id,
#    resource_group=resource_group, 
#    location=workspace_region,
#    create_resource_group=True,
#    sku='basic',
#    exist_ok=True
#)
#ws.get_details()

## 2.0 Write config file
Write the details of the workspace to a config.json file:

In [ ]:
ws.write_config()